In [2]:
import torch

## Setup

In [3]:
inputs = torch.tensor(
    [
        [0.43, 0.15, 0.89],  # Your     (x^1)
        [0.55, 0.87, 0.66],  # journey  (x^2)
        [0.57, 0.85, 0.64],  # starts   (x^3)
        [0.22, 0.58, 0.33],  # with     (x^4)
        [0.77, 0.25, 0.10],  # one      (x^5)
        [0.05, 0.80, 0.55],
    ]  # step     (x^6)
)

In [4]:
query = inputs[1]  # doesn't need to be like this - just to simplify

## Simplified Self-Attention

In [5]:
attn_scores_2 = torch.empty(inputs.shape[0])

In [6]:
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(
        x_i, query
    )  # dot product measures similarity between two vectors
print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [7]:
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()
print("Attention weights:", attn_weights_2_tmp)
print("Sum:", attn_weights_2_tmp.sum())

Attention weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: tensor(1.0000)


In [8]:
# actually, it's better to use softmax
def naive_softmax(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

In [9]:
attn_weights_2_naive = naive_softmax(attn_scores_2)
print("Attention weights (naive softmax):", attn_weights_2_naive)
print("Sum (naive softmax):", attn_weights_2_naive.sum())

Attention weights (naive softmax): tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum (naive softmax): tensor(1.)


In [10]:
# even better to use torch's built-in softmax
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)
print("Attention weights (torch softmax):", attn_weights_2)
print("Sum (torch softmax):", attn_weights_2.sum())

Attention weights (torch softmax): tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum (torch softmax): tensor(1.)


In [11]:
# final step - calculate context vector
context_vec_2 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i] * x_i
print("Context vector:", context_vec_2)

Context vector: tensor([0.4419, 0.6515, 0.5683])


In [12]:
# now, for the full attention matrix
attn_scores = torch.empty(6, 6)
for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)
print("Attention scores matrix:\n", attn_scores)

Attention scores matrix:
 tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [13]:
# better to use @ operator
attn_scores = inputs @ inputs.T
print("Attention scores matrix (using @):\n", attn_scores)

Attention scores matrix (using @):
 tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [14]:
# and now for the normalization
attn_weights = torch.softmax(attn_scores, dim=-1)
print("Attention weights matrix:\n", attn_weights)

Attention weights matrix:
 tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [15]:
all_context_vecs = attn_weights @ inputs
print("All context vectors:\n", all_context_vecs)

All context vectors:
 tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


## Self-Attention

... also called _scaled dot-product attention_

In [16]:
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2

In [17]:
torch.manual_seed(42)
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

In [18]:
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value
print("Query vector:", query_2)

Query vector: tensor([1.0760, 1.7344])


In [19]:
keys = inputs @ W_key
values = inputs @ W_value
print("Keys matrix:\n", keys)
print("Values matrix:\n", values)

Keys matrix:
 tensor([[1.0832, 0.8895],
        [1.5764, 0.9441],
        [1.5440, 0.9455],
        [0.9105, 0.4477],
        [0.5262, 0.7038],
        [1.2795, 0.4727]])
Values matrix:
 tensor([[1.2731, 0.8193],
        [1.7073, 1.0646],
        [1.6922, 1.0559],
        [0.9133, 0.5633],
        [0.9433, 0.6019],
        [1.1233, 0.6876]])


In [20]:
keys_2 = keys[1]
attn_scores_22 = query_2.dot(keys_2)
print("Attention score:", attn_scores_22)

Attention score: tensor(3.3338)


In [21]:
attn_scores_2 = query_2 @ keys.T
print("Attention scores:", attn_scores_2)

Attention scores: tensor([2.7084, 3.3338, 3.3013, 1.7563, 1.7869, 2.1966])


In [22]:
d_k = keys.shape[-1]
attns_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)
print("Attention weights:", attns_weights_2)

Attention weights: tensor([0.1723, 0.2681, 0.2620, 0.0879, 0.0898, 0.1200])


In [23]:
# and now the context vector
context_vec_2 = attns_weights_2 @ values
print("Context vector:", context_vec_2)

Context vector: tensor([1.4201, 0.8892])


In [24]:
import os

os.chdir("../..")

from theory.attention.attention import SelfAttention_v1, SelfAttention_v2

In [25]:
torch.manual_seed(42)
sa_v1 = SelfAttention_v1(d_in, d_out)
context_vecs = sa_v1(inputs)
print("All context vectors (using SelfAttention_v1):\n", context_vecs)

All context vectors (using SelfAttention_v1):
 tensor([[0.5141, 0.3639],
        [0.5633, 0.3251],
        [0.5659, 0.3221],
        [0.5839, 0.2941],
        [0.6180, 0.2539],
        [0.5575, 0.3262]], grad_fn=<MmBackward0>)


In [26]:
torch.manual_seed(42)
sa_v2 = SelfAttention_v2(d_in, d_out)
context_vecs = sa_v2(inputs)
print("All context vectors (using SelfAttention_v2):\n", context_vecs)

All context vectors (using SelfAttention_v2):
 tensor([[0.3755, 0.2777],
        [0.3761, 0.2831],
        [0.3761, 0.2833],
        [0.3768, 0.2763],
        [0.3754, 0.2836],
        [0.3772, 0.2746]], grad_fn=<MmBackward0>)


In [ ]:
# TODO: implement task

# We can transfer the weight matrices from a SelfAttention_v2 object to a Self-
# Attention_v1, such that both objects then produce the same results.
# Your task is to correctly assign the weights from an instance of SelfAttention_v2
# to an instance of SelfAttention_v1. To do this, you need to understand the rela-
# tionship between the weights in both versions. (Hint: nn.Linear stores the weight
# matrix  in  a  transposed  form.)  After  the  assignment,  you  should  observe  that  both
# instances produce the same outputs.